In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import sys

In [25]:
import sys
sys.path.append('../scr')

from utils_clean import filtrar_y_crear_features_semanal, partir_train_test
from utils_data import downloadTable
from utils_model import buscar_hiperparametros_arima_sarima
from utils_metrics import errorMetrics

In [26]:
queryPrices = """
SELECT * FROM `desarrollo-444913.globalPrices.prices`
"""
# Usar forceDownload=True para actualizar la tabla
df = downloadTable(queryPrices,"globalPricesTable",forceDownload=False)

Folder 'bigqueryDatabases' already exists.
Reading bigqueryDatabases\globalPricesTable.csv from local CSV.


In [27]:
df

,date,price,priceId,priceName,currency,currencyId,priceType,source
0,2022-10-01,14.637188,4,Japan Frozen Atlantic TRIM-E,USD,0,EXPORT,CHILEAN EXPORTS
1,2023-03-01,12.265160,5,Japan Fresh Atlantic TRIM-A,USD,0,EXPORT,CHILEAN EXPORTS
2,2024-02-01,5.859417,1,Japan Frozen Coho HG,USD,0,EXPORT,CHILEAN EXPORTS
3,2012-11-08,6.724091,11,UB Atlantic TRIM-D 3-4 Lb FOB Miami,USD,0,EXPORTS,URNER BARRY
4,2007-12-26,7.495708,11,UB Atlantic TRIM-D 3-4 Lb FOB Miami,USD,0,EXPORTS,URNER BARRY
...,...,...,...,...,...,...,...,...
8141,2025-06-03,NaN,13,UB Canada NE Whole 10-12 Lb FOB Miami,USD,0,EXPORTS,URNER BARRY
8142,2025-04-01,13.406630,7,Japan Frozen Trout TRIM-E,USD,0,EXPORT,CHILEAN EXPORTS
8143,2025-04-01,11.869333,8,Japan Frozen Trout TRIM-C,USD,0,EXPORT,CHILEAN EXPORTS
8144,2025-04-01,6.280960,9,Japan Frozen Trout HG,USD,0,EXPORT,CHILEAN EXPORTS


In [28]:

df_1 = df[df['priceName']=="UB Atlantic TRIM-D 3-4 Lb FOB Miami"]  
df_1['date'] = pd.to_datetime(df_1['date'])
df_1['iso_year'] = df_1['date'].dt.isocalendar().year
df_1['iso_week'] = df_1['date'].dt.isocalendar().week

# Cuenta cuántos prices hay por semana
conteo_semanal = df_1.groupby(['iso_year', 'iso_week'])['price'].count().reset_index()
conteo_semanal = conteo_semanal.rename(columns={'price': 'count_prices'})

print(conteo_semanal)


     iso_year  iso_week  count_prices
0        2007         1             2
1        2007         2             2
2        2007         3             2
3        2007         4             2
4        2007         5             2
..        ...       ...           ...
957      2025        19             2
958      2025        20             2
959      2025        21             2
960      2025        22             2
961      2025        23             2

[962 rows x 3 columns]


In [29]:
# quiero ver cuántos valores de 0 y 1 hay en count_prices
conteo_semanal['count_prices'].value_counts()

count_prices
2    954
1      4
0      4
Name: count, dtype: int64

# Preprocesamiento

In [30]:
df = filtrar_y_crear_features_semanal(
    df,
    weeks_lags    = [1, 2, 3, 4],    # retardos de 1 a 4 semanas
    weeks_windows = [4, 12],         # medias móviles de 4 y 12 semanas
    obs_per_week  = 2                # porque recibes 2 precios/semana
)

In [31]:
df

,price,year,weekofyear,lag_1wk,lag_2wk,lag_3wk,lag_4wk,ma_4wk,ma_12wk
date,,,,,,,,,
2007-06-18,8.212209,2007,25,8.873595,9.094057,9.149173,9.314519,8.908043,9.103243
2007-06-25,7.936632,2007,26,8.598018,9.038942,9.149173,9.149173,8.756475,9.057314
2007-07-02,7.881516,2007,27,8.212209,8.873595,9.094057,9.149173,8.598018,9.009088
2007-07-09,7.826401,2007,28,7.936632,8.598018,9.038942,9.149173,8.432671,8.951676
2007-07-16,7.826401,2007,29,7.881516,8.212209,8.873595,9.094057,8.274214,8.887374
...,...,...,...,...,...,...,...,...,...
2025-05-12,14.054452,2025,20,14.605607,14.109568,13.613528,13.999337,14.082010,13.794950
2025-05-19,13.393066,2025,21,14.605607,14.440261,13.337951,13.889106,14.020005,13.815619
2025-05-26,13.007258,2025,22,14.054452,14.605607,14.109568,13.613528,13.944221,13.822508


# División train test

In [32]:
fecha_entrenamiento_fin = '2023-12-31'
fecha_prueba_ini       = '2024-01-01'

train = df.loc[:fecha_entrenamiento_fin]
test  = df.loc[fecha_prueba_ini:]

X_train = train.drop(columns=['price'])
y_train = train['price']
X_test  = test.drop(columns=['price'])
y_test  = test['price']


## Entrenamiento

In [33]:
# usar XGBoost regressor para predecir el precio
# buscar hiperparámetros
